In [44]:
import tweepy
import pandas as pd 
import json
from datetime import datetime
import s3fs

In [2]:
import configparser

In [3]:
config = configparser.ConfigParser()

In [22]:
config.read('clustwiter.config')

['clustwiter.config']

**API key and secret:**

Think of these as the user name and password that represents your Twitter developer app when making API requests. 

oauth_consumer_key

oauth_consumer_secret


**Access token and secret:**
An access token and access token secret are user-specific credentials used to authenticate OAuth 1.0a API requests. They specify the Twitter account the request is made on behalf of.

You can generate your own access token and token secret if you would like your app to make requests on behalf of the same Twitter account associated with your developer account on the Twitter developer app's details page.

oauth_token

oauth_token_secret


In [29]:
access_key = config['TWITTER']['access_token'] 
access_secret = config['TWITTER']['access_secret']
consumer_key = config['TWITTER']['api_key']
consumer_secret = config['TWITTER']['api_secret']

In [35]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

In [37]:
auth.set_access_token(access_key, access_secret)

In [38]:
api = tweepy.API(auth)

In [41]:
def run_twitter_etl():
    
    access_key = config['TWITTER']['access_token'] 
    access_secret = config['TWITTER']['access_secret']
    consumer_key = config['TWITTER']['api_key']
    consumer_secret = config['TWITTER']['api_secret']


    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)

    # # # Creating an API object 
    api = tweepy.API(auth)
    tweets = api.user_timeline(screen_name='@elonmusk', 
                            # 200 is the maximum allowed count
                            count=50,
                            include_rts = False,
                            # Necessary to keep full_text 
                            # otherwise only the first 140 words are extracted
                            tweet_mode = 'extended'
                            )

    print(tweets[0])
    list = []
    for tweet in tweets:
        text = tweet._json["full_text"]

        refined_tweet = {"user": tweet.user.screen_name,
                        'text' : text,
                        'favorite_count' : tweet.favorite_count,
                        'retweet_count' : tweet.retweet_count,
                        'created_at' : tweet.created_at}
        
        list.append(refined_tweet)

    df = pd.DataFrame(list)
    df.to_csv('refined_tweets.csv')

In [43]:
from datetime import timedelta
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.utils.dates import days_ago
from datetime import datetime
from twitter_etl import run_twitter_etl

In [19]:
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime(2020, 11, 8),
    'email': ['airflow@example.com'],
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=1)
}

In [22]:
dag = DAG(
    'twitter_dag',
    default_args=default_args,
    description='Our first DAG with ETL process!',
    schedule=timedelta(days=1),
)

In [23]:
dag = DAG(
    'twitter_dag',
    default_args=default_args,
    description='My first etl code'
)


In [24]:
run_etl = PythonOperator(
    task_id='complete_twitter_etl',
    python_callable=run_twitter_etl,
    dag=dag, 
)